## 라이브러리

In [17]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import warnings
warnings.filterwarnings('ignore')

## path 설정

In [18]:
path = '../..'
# path = 
script_path = f'{path}/script/csv_files'

## 파일 불러오기

In [19]:
df = pd.read_csv(f'{script_path}/all_scripts_normalization.csv').set_index('title')
df.head()

,level_1,level_2,level_3,level_4,level_5,level_6,WPS_mean,WPS_std,WPS_min,WPS_max,WPS_mean_nlln
title,,,,,,,,,,,
13TH,0.609863,0.763039,0.611899,0.846982,0.377385,0.426996,2.961973,0.793955,1.065530,5.997001,0.516759
About.Time,0.374061,0.174619,0.147663,0.210125,0.186990,0.094885,2.182928,1.094544,0.503440,6.080000,0.081738
Arthur.Christmas,0.325880,0.154821,0.198371,0.827417,0.448185,0.400896,2.156860,0.925752,0.510204,5.263158,0.067182
Baby.Driver,0.330802,0.123976,0.254857,0.441518,0.150633,0.356591,2.833974,1.249704,0.574053,10.657194,0.445284
Bad.Boys,0.473384,0.202440,0.285286,0.544960,0.410822,0.320087,2.507442,1.155209,0.538648,7.052186,0.262948


## level 별 가중치 계산

In [20]:
df_result = df.copy()
level_1 = df_result['level_1']
level_2 = df_result['level_2']
level_3 = df_result['level_3']
level_4 = df_result['level_4']
level_5 = df_result['level_5']
level_6 = df_result['level_6']

df_result['level_sum'] = level_1 + level_2 + level_3 + level_4 + level_5 + level_6
df_result['80%'] = df_result['level_sum'] * 0.8
df_result


,level_1,level_2,level_3,level_4,level_5,level_6,WPS_mean,WPS_std,WPS_min,WPS_max,WPS_mean_nlln,level_score,80%
title,,,,,,,,,,,,,
13TH,0.609863,0.763039,0.611899,0.846982,0.377385,0.426996,2.961973,0.793955,1.065530,5.997001,0.516759,3.636163,2.908931
About.Time,0.374061,0.174619,0.147663,0.210125,0.186990,0.094885,2.182928,1.094544,0.503440,6.080000,0.081738,1.188342,0.950674
Arthur.Christmas,0.325880,0.154821,0.198371,0.827417,0.448185,0.400896,2.156860,0.925752,0.510204,5.263158,0.067182,2.355570,1.884456
Baby.Driver,0.330802,0.123976,0.254857,0.441518,0.150633,0.356591,2.833974,1.249704,0.574053,10.657194,0.445284,1.658376,1.326701
Bad.Boys,0.473384,0.202440,0.285286,0.544960,0.410822,0.320087,2.507442,1.155209,0.538648,7.052186,0.262948,2.236981,1.789585
Bee.Movie,0.474071,0.364282,0.370651,0.558443,0.270133,0.648519,2.617452,1.143877,0.515597,6.216696,0.324377,2.686098,2.148879
Ben.Platt.Live.from.Radio.City.Music.Hall,0.098472,0.000000,0.009014,0.007444,0.065981,0.150052,3.827371,1.111227,1.141553,6.657790,1.000000,0.330963,0.264770
Bo.Burnham_.Inside,0.058552,0.054582,0.000000,0.000000,0.046040,0.078117,2.763471,0.968897,0.677535,5.289140,0.405915,0.237291,0.189833
Bo.Burnham_.Make.Happy,0.223802,0.112670,0.171000,0.197196,0.303226,0.323077,3.410115,1.234155,0.773395,7.380074,0.767003,1.330971,1.064777


## level들을 합한 값의 80% 위치에 해당하는 level 설정

In [21]:
def level_result(_list, percentile):
    sum = 0
    count = 1
    for level in _list:
        sum += level
        if sum > percentile:
            break
        count += 1
    return count
    
df_result['word_level'] = df_result.apply(lambda x: level_result([x['level_1'], x['level_2'], x['level_3'], x['level_4'], x['level_5'], x['level_6']], x['80%']), axis=1)
df_result
# def level_result(ratio):
#     if ratio <= 16.6:
#         result = 1
#     elif ratio <= 33.2:
#         result = 2
#     elif ratio <= 49.8:
#         result = 3
#     elif ratio <= 66.4:
#         result = 4
#     elif ratio <= 83:
#         result = 5
#     else:
#         result = 6

#     return result

# df_result['word_level'] = df_result.apply(lambda x: level_result(x['ratio']), axis=1)
# df_result

,level_1,level_2,level_3,level_4,level_5,level_6,WPS_mean,WPS_std,WPS_min,WPS_max,WPS_mean_nlln,level_score,80%,word_level
title,,,,,,,,,,,,,,
13TH,0.609863,0.763039,0.611899,0.846982,0.377385,0.426996,2.961973,0.793955,1.065530,5.997001,0.516759,3.636163,2.908931,5
About.Time,0.374061,0.174619,0.147663,0.210125,0.186990,0.094885,2.182928,1.094544,0.503440,6.080000,0.081738,1.188342,0.950674,5
Arthur.Christmas,0.325880,0.154821,0.198371,0.827417,0.448185,0.400896,2.156860,0.925752,0.510204,5.263158,0.067182,2.355570,1.884456,5
Baby.Driver,0.330802,0.123976,0.254857,0.441518,0.150633,0.356591,2.833974,1.249704,0.574053,10.657194,0.445284,1.658376,1.326701,6
Bad.Boys,0.473384,0.202440,0.285286,0.544960,0.410822,0.320087,2.507442,1.155209,0.538648,7.052186,0.262948,2.236981,1.789585,5
Bee.Movie,0.474071,0.364282,0.370651,0.558443,0.270133,0.648519,2.617452,1.143877,0.515597,6.216696,0.324377,2.686098,2.148879,6
Ben.Platt.Live.from.Radio.City.Music.Hall,0.098472,0.000000,0.009014,0.007444,0.065981,0.150052,3.827371,1.111227,1.141553,6.657790,1.000000,0.330963,0.264770,6
Bo.Burnham_.Inside,0.058552,0.054582,0.000000,0.000000,0.046040,0.078117,2.763471,0.968897,0.677535,5.289140,0.405915,0.237291,0.189833,6
Bo.Burnham_.Make.Happy,0.223802,0.112670,0.171000,0.197196,0.303226,0.323077,3.410115,1.234155,0.773395,7.380074,0.767003,1.330971,1.064777,6


In [22]:
df_result = df_result.drop(columns=['80%'])
df_result

,level_1,level_2,level_3,level_4,level_5,level_6,WPS_mean,WPS_std,WPS_min,WPS_max,WPS_mean_nlln,level_score,word_level
title,,,,,,,,,,,,,
13TH,0.609863,0.763039,0.611899,0.846982,0.377385,0.426996,2.961973,0.793955,1.065530,5.997001,0.516759,3.636163,5
About.Time,0.374061,0.174619,0.147663,0.210125,0.186990,0.094885,2.182928,1.094544,0.503440,6.080000,0.081738,1.188342,5
Arthur.Christmas,0.325880,0.154821,0.198371,0.827417,0.448185,0.400896,2.156860,0.925752,0.510204,5.263158,0.067182,2.355570,5
Baby.Driver,0.330802,0.123976,0.254857,0.441518,0.150633,0.356591,2.833974,1.249704,0.574053,10.657194,0.445284,1.658376,6
Bad.Boys,0.473384,0.202440,0.285286,0.544960,0.410822,0.320087,2.507442,1.155209,0.538648,7.052186,0.262948,2.236981,5
Bee.Movie,0.474071,0.364282,0.370651,0.558443,0.270133,0.648519,2.617452,1.143877,0.515597,6.216696,0.324377,2.686098,6
Ben.Platt.Live.from.Radio.City.Music.Hall,0.098472,0.000000,0.009014,0.007444,0.065981,0.150052,3.827371,1.111227,1.141553,6.657790,1.000000,0.330963,6
Bo.Burnham_.Inside,0.058552,0.054582,0.000000,0.000000,0.046040,0.078117,2.763471,0.968897,0.677535,5.289140,0.405915,0.237291,6
Bo.Burnham_.Make.Happy,0.223802,0.112670,0.171000,0.197196,0.303226,0.323077,3.410115,1.234155,0.773395,7.380074,0.767003,1.330971,6


## WPS Score 계산

In [23]:
df_result['wps_score'] = df_result['WPS_mean_nlln']
df_result

,level_1,level_2,level_3,level_4,level_5,level_6,WPS_mean,WPS_std,WPS_min,WPS_max,WPS_mean_nlln,level_score,word_level,wps_score
title,,,,,,,,,,,,,,
13TH,0.609863,0.763039,0.611899,0.846982,0.377385,0.426996,2.961973,0.793955,1.065530,5.997001,0.516759,3.636163,5,0.516759
About.Time,0.374061,0.174619,0.147663,0.210125,0.186990,0.094885,2.182928,1.094544,0.503440,6.080000,0.081738,1.188342,5,0.081738
Arthur.Christmas,0.325880,0.154821,0.198371,0.827417,0.448185,0.400896,2.156860,0.925752,0.510204,5.263158,0.067182,2.355570,5,0.067182
Baby.Driver,0.330802,0.123976,0.254857,0.441518,0.150633,0.356591,2.833974,1.249704,0.574053,10.657194,0.445284,1.658376,6,0.445284
Bad.Boys,0.473384,0.202440,0.285286,0.544960,0.410822,0.320087,2.507442,1.155209,0.538648,7.052186,0.262948,2.236981,5,0.262948
Bee.Movie,0.474071,0.364282,0.370651,0.558443,0.270133,0.648519,2.617452,1.143877,0.515597,6.216696,0.324377,2.686098,6,0.324377
Ben.Platt.Live.from.Radio.City.Music.Hall,0.098472,0.000000,0.009014,0.007444,0.065981,0.150052,3.827371,1.111227,1.141553,6.657790,1.000000,0.330963,6,1.000000
Bo.Burnham_.Inside,0.058552,0.054582,0.000000,0.000000,0.046040,0.078117,2.763471,0.968897,0.677535,5.289140,0.405915,0.237291,6,0.405915
Bo.Burnham_.Make.Happy,0.223802,0.112670,0.171000,0.197196,0.303226,0.323077,3.410115,1.234155,0.773395,7.380074,0.767003,1.330971,6,0.767003


## Final Score 계산

In [24]:
df_result['final_score'] = df_result['level_score'] + df_result['wps_score']
df_result

,level_1,level_2,level_3,level_4,level_5,level_6,WPS_mean,WPS_std,WPS_min,WPS_max,WPS_mean_nlln,level_score,word_level,wps_score,final_score
title,,,,,,,,,,,,,,,
13TH,0.609863,0.763039,0.611899,0.846982,0.377385,0.426996,2.961973,0.793955,1.065530,5.997001,0.516759,3.636163,5,0.516759,4.152923
About.Time,0.374061,0.174619,0.147663,0.210125,0.186990,0.094885,2.182928,1.094544,0.503440,6.080000,0.081738,1.188342,5,0.081738,1.270080
Arthur.Christmas,0.325880,0.154821,0.198371,0.827417,0.448185,0.400896,2.156860,0.925752,0.510204,5.263158,0.067182,2.355570,5,0.067182,2.422751
Baby.Driver,0.330802,0.123976,0.254857,0.441518,0.150633,0.356591,2.833974,1.249704,0.574053,10.657194,0.445284,1.658376,6,0.445284,2.103660
Bad.Boys,0.473384,0.202440,0.285286,0.544960,0.410822,0.320087,2.507442,1.155209,0.538648,7.052186,0.262948,2.236981,5,0.262948,2.499928
Bee.Movie,0.474071,0.364282,0.370651,0.558443,0.270133,0.648519,2.617452,1.143877,0.515597,6.216696,0.324377,2.686098,6,0.324377,3.010475
Ben.Platt.Live.from.Radio.City.Music.Hall,0.098472,0.000000,0.009014,0.007444,0.065981,0.150052,3.827371,1.111227,1.141553,6.657790,1.000000,0.330963,6,1.000000,1.330963
Bo.Burnham_.Inside,0.058552,0.054582,0.000000,0.000000,0.046040,0.078117,2.763471,0.968897,0.677535,5.289140,0.405915,0.237291,6,0.405915,0.643207
Bo.Burnham_.Make.Happy,0.223802,0.112670,0.171000,0.197196,0.303226,0.323077,3.410115,1.234155,0.773395,7.380074,0.767003,1.330971,6,0.767003,2.097974


## csv 파일로 변환

In [ ]:
df_result.to_csv(f'{script_path}/all_scripts_final_score.csv')